# フィーチャースケーリング - 課題の回答

対象とするデータが入っているファイルは: 'real_estate_price_size_year.csv'です

データをまずは標準化しましょう

その上で:
-  係数と切片を表示しましょう
-  決定係数と自由度修正済み決定係数を求めましょう
-  決定係数と自由度修正済み決定係数を比較しましょう
-  単回帰分析の場合における決定係数と比較しましょう
-  サイズが750の場合における値段を予測しましょう
-  二つの変数それぞれのp値を求めましょう
-  まとめの表を作成しましょう

## ライブラリのインポート

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression

## データの読み込み

In [4]:
data = pd.read_csv('real_estate_price_size_year.csv')
data.head()

,price,size,year
0,234314.144,643.09,2015
1,228581.528,656.22,2009
2,281626.336,487.29,2018
3,401255.608,1504.75,2015
4,458674.256,1275.46,2009


In [6]:
data.describe()

,price,size,year
count,100.000000,100.000000,100.000000
mean,292289.470160,853.024200,2012.600000
std,77051.727525,297.941951,4.729021
min,154282.128000,479.750000,2006.000000
25%,234280.148000,643.330000,2009.000000
50%,280590.716000,696.405000,2015.000000
75%,335723.696000,1029.322500,2018.000000
max,500681.128000,1842.510000,2018.000000


## 回帰モデルの作成

### 従属変数と独立変数の宣言

In [8]:
x = data[['size','year']]
y = data['price']

### 入力の標準化

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)

### 回帰

In [12]:
reg = LinearRegression()
reg.fit(x_scaled,y)

LinearRegression()

### 係数の表示

In [14]:
reg.intercept_

292289.4701599997

### 切片の表示

In [16]:
reg.coef_

array([67501.57614152, 13724.39708231])

### 決定係数の計算

In [18]:
reg.score(x_scaled,y)

0.7764803683276793

### 自由度修正済み決定係数の計算

In [20]:
# 以下が自由度修正済み決定係数を求める式です
def adj_r2(x,y):
    r2 = reg.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [22]:
adj_r2(x_scaled,y)

0.77187171612825

### 決定係数と自由度修正済み決定係数の比較

両者の値にそこまで違いはないことが分かります。つまり、変数を増やすことによるペナルティは小さいことが分かります

### 自由度修正済み決定係数と単回帰の場合の決定係数との比較

両者の値を比較すると、Yearに関する情報は結果に対してほとんど影響を与えていないことが分かります

### 予測

In [24]:
new_data = [[750,2009]]
new_data_scaled = scaler.transform(new_data)

In [26]:
reg.predict(new_data_scaled)

array([258330.34465995])

### それぞの変数のp値の計算

In [28]:
from sklearn.feature_selection import f_regression

In [30]:
f_regression(x_scaled,y)

(array([285.92105192,   0.85525799]), array([8.12763222e-31, 3.57340758e-01]))

In [32]:
p_values = f_regression(x,y)[1]
p_values

array([8.12763222e-31, 3.57340758e-01])

In [34]:
p_values.round(3)

array([0.   , 0.357])

### まとめ表の作成

In [36]:
reg_summary = pd.DataFrame(data = x.columns.values, columns=['Features'])
reg_summary ['Coefficients'] = reg.coef_
reg_summary ['p-values'] = p_values.round(3)
reg_summary

,Features,Coefficients,p-values
0,size,67501.576142,0.000
1,year,13724.397082,0.357


Yearはモデルに対して有意な影響を持っていないことが分かりますので、削除しても良いとみなすことができます